In [1]:
import gurobipy as gp
from gurobipy import *
import pandas as pd
import numpy as np

In [2]:
# Specify data types for problematic columns
dtype_options = {'nation_position': 'str', 'nation_logo_url': 'str'}

players = pd.read_csv('players_22.csv', dtype=dtype_options)

In [3]:
players.shape

(19239, 110)

In [4]:
players.loc[:, 'pace': 'goalkeeping_speed'].isnull().sum()

pace                            2132
shooting                        2132
passing                         2132
dribbling                       2132
defending                       2132
physic                          2132
attacking_crossing                 0
attacking_finishing                0
attacking_heading_accuracy         0
attacking_short_passing            0
attacking_volleys                  0
skill_dribbling                    0
skill_curve                        0
skill_fk_accuracy                  0
skill_long_passing                 0
skill_ball_control                 0
movement_acceleration              0
movement_sprint_speed              0
movement_agility                   0
movement_reactions                 0
movement_balance                   0
power_shot_power                   0
power_jumping                      0
power_stamina                      0
power_strength                     0
power_long_shots                   0
mentality_aggression               0
m

In [5]:
#players.dropna(subset=players.loc[:, 'pace': 'goalkeeping_reflexes'].columns, inplace=True)
# We don't drop missing values because some missing values are associated with positions. For example, goalkeepers don't have scores for pace, shooting...

In [6]:
# Separate the position score columns, for example, column 'ls' with value '83+3' separates into column 'ls' with value 83 and column 'ls_potential' 3
for col in players.loc[:, 'ls': 'gk'].columns:
    # Extract the numbers using regular expressions and handle the case where the sign is not present
    extracted_values = players[col].str.extract(r'(\d+)([\+\-]?\d*)')
    # Separate the extracted values into two columns
    players[[col, col+'_potential']] = extracted_values.apply(lambda x: pd.to_numeric(x, errors='coerce'))
    # Fill missing values with 0 in the 'Value2' column
    players[col+'_potential'].fillna(0, inplace=True)

In [7]:
players['player_positions'].isnull().sum()

0

In [8]:
# Cleaning the "player_positions" column by removing quotes
players['player_positions'] = players['player_positions'].str.replace('"', '')
players['player_positions'] = players['player_positions'].str.replace(' ', '')

# Splitting the positions and getting dummies
positions_dummies = players['player_positions'].str.get_dummies(sep=',')

# Merging the dummies back into the original dataset
players = pd.concat([players, positions_dummies], axis=1)

#players.to_csv('~/Desktop/players.csv', index=False)

In [9]:
# Creating the 'forward_score' column
forward_cols = ['pace', 'shooting', 'passing', 'dribbling', 'physic', 'attacking_crossing',
                'attacking_finishing', 'attacking_heading_accuracy', 'attacking_volleys',
                'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
                'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed',
                'movement_agility', 'movement_reactions', 'power_shot_power', 'power_jumping',
                'power_strength', 'power_long_shots', 'mentality_aggression',
                'mentality_interceptions', 'mentality_positioning', 'mentality_vision',
                'mentality_penalties', 'mentality_composure']

players['forward_score'] = players[forward_cols].sum(axis=1)

# Creating the 'mid_score' column
mid_cols = ['pace', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing',
            'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve',
            'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_agility',
            'movement_balance', 'mentality_aggression', 'mentality_interceptions',
            'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure',
            'defending_marking_awareness']

players['mid_score'] = players[mid_cols].sum(axis=1)

# Creating the 'back_score' column
back_cols = ['passing', 'defending', 'physic', 'movement_acceleration', 'movement_sprint_speed',
             'movement_agility', 'movement_reactions', 'movement_balance', 'power_jumping',
             'power_stamina', 'power_strength', 'mentality_positioning', 'mentality_vision',
             'mentality_composure', 'defending_marking_awareness', 'defending_standing_tackle',
             'defending_sliding_tackle']

players['back_score'] = players[back_cols].sum(axis=1)

In [10]:
#pd.set_option('display.max_columns', 150)
#players.head()

In [11]:
players_data = players[players['nationality_name'] == 'China PR']
players_data.shape

(385, 155)

In [13]:
pd.set_option('display.max_columns', 150)
players_data.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,...,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,ls_potential,st_potential,rs_potential,lw_potential,lf_potential,cf_potential,rf_potential,rw_potential,lam_potential,cam_potential,ram_potential,lm_potential,lcm_potential,cm_potential,rcm_potential,rm_potential,lwb_potential,ldm_potential,cdm_potential,rdm_potential,rwb_potential,lb_potential,lcb_potential,cb_potential,rcb_potential,rb_potential,gk_potential,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST,forward_score,mid_score,back_score
536,192064,https://sofifa.com/player/192064/late-gao/220002,Gao Late,高拉特,"ST,RW,CAM",79,79,17000000.0,32000.0,30,1991-06-05,185,78,111839.0,Guangzhou FC,Chinese Super League,1.0,CAM,11.0,NaN,2020-01-01,2023.0,155,China PR,NaN,NaN,NaN,Right,4,4,2,High/Medium,Normal (185+),No,27200000.0,NaN,"Power Free-Kick, Playmaker (AI), Chip Shot (AI...",83.0,78.0,77.0,79.0,41.0,72.0,73,77,78,81,77,78,70,72,74,83,82,83,74,76,81,78,78,76,73,79,62,33,83,77,76,78,37,41,34,9,14,8,...,79,79,80,80,80,79,79,79,79,79,75,75,75,79,62,61,61,61,62,59,54,54,54,59,17,https://cdn.sofifa.net/players/192/064/22_120.png,https://cdn.sofifa.net/teams/111839/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,2103.0,1561.0,1163.0
538,192598,https://sofifa.com/player/192598/kesen-ai/220002,Ai Kesen,艾克森,ST,79,79,15000000.0,32000.0,31,1989-07-13,180,77,111839.0,Guangzhou FC,Chinese Super League,1.0,RS,9.0,NaN,2019-07-09,2023.0,155,China PR,1413.0,ST,9.0,Right,4,4,2,High/High,Stocky (170-185),No,24000000.0,NaN,"Power Free-Kick, Chip Shot (AI), Technical Dri...",84.0,80.0,73.0,79.0,48.0,79.0,67,80,72,75,76,78,76,80,66,79,87,81,83,75,90,86,83,81,82,72,69,56,77,78,85,76,43,45,35,9,16,7,...,79,78,78,78,78,78,77,77,77,77,73,73,73,77,65,64,64,64,65,62,59,59,59,62,17,https://cdn.sofifa.net/players/192/598/22_120.png,https://cdn.sofifa.net/teams/111839/60.png,https://cdn.sofifa.net/flags/cn.png,https://cdn.sofifa.net/teams/1413/60.png,https://cdn.sofifa.net/flags/cn.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2159.0,1617.0,1216.0
891,188044,https://sofifa.com/player/188044/lan-a/220002,A Lan,阿兰,"ST,LW,LM",77,77,9000000.0,27000.0,31,1989-07-10,178,64,111839.0,Guangzhou FC,Chinese Super League,1.0,SUB,18.0,NaN,2015-12-01,2023.0,155,China PR,1413.0,LW,11.0,Right,3,4,2,High/Low,Lean (170-185),No,14400000.0,NaN,"Long Passer (AI), Speed Dribbler (AI)",84.0,75.0,68.0,81.0,33.0,72.0,66,79,76,68,76,83,78,72,63,79,84,84,88,75,75,74,82,77,75,68,55,33,78,72,76,70,35,20,22,11,7,14,...,7

In [17]:
# Initialize the model
model = gp.Model("soccer_team_optimization")
model.Params.LogToConsole = 0

# Add variables: One for each player, 1 if the player is selected for 23-player team(including substitute), otherwise 0
X = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="player1/23")

# Add variables: One for each player, 1 if the player is selected for 11-player team(excluding substitute), otherwise 0
Y = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="player1/11")

# Forward: CF, ST
# Midfielder: CAM, CDM, CM, LM, LW, LWB, RM, RW, RWB
# Back: CB, LB, RB
# One player can have up to 3 positions

# Add variables: One for each player, 1/2/3 if the player is selected for 23-player team and is Forward/Midfielder/Back position, otherwise 0
Ft = model.addVars(players_data['sofifa_id'], vtype=GRB.INTEGER, name="Forward temp")
Mt = model.addVars(players_data['sofifa_id'], vtype=GRB.INTEGER, name="Midfielder temp")
Bt = model.addVars(players_data['sofifa_id'], vtype=GRB.INTEGER, name="Back temp")

# Add variables: One for each player, 1 if the player is selected for 23-player team and is Forward/Midfielder/Back position, otherwise 0
F = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="Forward")
M = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="Midfielder")
B = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="Back")

# Auxiliary variables for age constraint
Z = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="z")

# Objective 1: Maximize overall rating for 23 players
overall_rating_23 = gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'overall'].values[0] 
                             for p in players_data['sofifa_id'])
model.setObjectiveN(overall_rating_23, index = 0, priority = 2)

# Objective 2: Maximize skill scores for each position
forward = gp.quicksum(F[p] * players_data.loc[players_data['sofifa_id'] == p, 'forward_score'].values[0]
                             for p in players_data['sofifa_id'])
mid = gp.quicksum(M[p] * players_data.loc[players_data['sofifa_id'] == p, 'mid_score'].values[0]
                             for p in players_data['sofifa_id'])
back = gp.quicksum(B[p] * players_data.loc[players_data['sofifa_id'] == p, 'forward_score'].values[0]
                             for p in players_data['sofifa_id'])
model.setObjectiveN(forward+mid+back, index = 1, priority = 1)

# Objective 3: Maximize overall rating for 11 players
overall_rating_11 = gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'overall'].values[0] 
                             for p in players_data['sofifa_id'])
model.setObjectiveN(overall_rating_11, index = 2, priority = 0)

# Objective 4: Maximize team chemistry (number of players from the same club)
# chemistry = gp.quicksum(X[p1] * X[p2] 
#                         for i, p1 in enumerate(players_data['sofifa_id']) 
#                         for p2 in players_data['sofifa_id'][i+1:] 
#                         if players_data.loc[players_data['sofifa_id'] == p1, 'club_team_id'].values[0] == 
#                            players_data.loc[players_data['sofifa_id'] == p2, 'club_team_id'].values[0])
# model.setObjectiveN(chemistry, index = 3, priority = 0)
# multi objective need to be linear

model.ModelSense = GRB.MAXIMIZE

# Constraint: 23 players including substitutes
model.addConstr(X.sum() == 23)
# Constraint: 11 players excluding substitutes
model.addConstr(Y.sum() == 11)
# Constraint: Relationship between 11 and 23 players
model.addConstrs(Y[p] <= X[p] for p in players_data['sofifa_id'])
# Constraint: for 23 players, each player's age <= 40
model.addConstrs(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'age'].values[0] <= 40
                             for p in players_data['sofifa_id'])
# Constraint: for 23 players, average player's age <= 30, >= 28
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'age'].values[0]
                            for p in players_data['sofifa_id']) >= 28*23)
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'age'].values[0]
                            for p in players_data['sofifa_id']) <= 30*23)
# Constraint: for 23 players, at least 3 players with age <= 23 need to be selected
n = len(players_data)
M = 100
model.addConstrs(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'age'].values[0] <= 23+M*Z[p]
                             for p in players_data['sofifa_id'])
model.addConstr(Z.sum() <= n-3)



# Optimize the model
model.optimize()

# Get the selected players
selected_players = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if X[p].x > 0]

print("Selected Players:", selected_players)

Selected Players: ['Gao Late', 'Ai Kesen', 'A Lan', 'Fei Nanduo', 'Wu Lei', 'Yan Junling', 'Cao Yunding', 'Zeng Cheng', 'Wang Dalei', 'Hao Junmin', 'Yu Hanchao', 'Li Xuepeng', 'Zhang Linpeng', 'Tang Miao', 'Liu Binbin', 'Ji Xiang', 'Jiang Zhipeng', 'Li Lei', 'Jiang Guangtai', 'Li Ke', 'Deng Hanwen', 'Ming Tian', 'Gao Zhunyi']
